In [1]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping
import tensorflow as tf



C:\Users\Ashish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
url = "https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/data/iris.csv"
df = pd.read_csv(io.StringIO(requests.get(url).content.decode('utf-8')), na_values=['NA','?'])

In [3]:
# Encode text index
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_

# convert a pandas dataframe to the x,y that tf needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find the type of the target column.
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type,'__iter__') else target_type
    print(target_type)
    # Encode to int for classification, float otherwise
    if target_type in (np.int64, np.int32):
        # classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

from sklearn import preprocessing
species = encode_text_index(df,'species')
x,y = to_xy(df, 'species')


int64


In [4]:

# model initialize and compile
model = Sequential()
model.add(Dense(50,
               input_dim=x.shape[1],
               activation = 'relu'))
model.add(Dense(25,
               activation='relu'))
model.add(Dense(y.shape[1],
               activation='softmax'))

model.compile(loss='categorical_crossentropy', 
             optimizer='adam')
model.fit(x,y,verbose=2,epochs=100)

Epoch 1/100
 - 2s - loss: 1.3031
Epoch 2/100
 - 0s - loss: 1.0448
Epoch 3/100
 - 0s - loss: 0.8951
Epoch 4/100
 - 0s - loss: 0.8523
Epoch 5/100
 - 0s - loss: 0.8027
Epoch 6/100
 - 0s - loss: 0.7476
Epoch 7/100
 - 0s - loss: 0.7057
Epoch 8/100
 - 0s - loss: 0.6665
Epoch 9/100
 - 0s - loss: 0.6365
Epoch 10/100
 - 0s - loss: 0.6028
Epoch 11/100
 - 0s - loss: 0.5748
Epoch 12/100
 - 0s - loss: 0.5514
Epoch 13/100
 - 0s - loss: 0.5291
Epoch 14/100
 - 0s - loss: 0.5093
Epoch 15/100
 - 0s - loss: 0.4903
Epoch 16/100
 - 0s - loss: 0.4724
Epoch 17/100
 - 0s - loss: 0.4568
Epoch 18/100
 - 0s - loss: 0.4426
Epoch 19/100
 - 0s - loss: 0.4282
Epoch 20/100
 - 0s - loss: 0.4152
Epoch 21/100
 - 0s - loss: 0.4048
Epoch 22/100
 - 0s - loss: 0.3921
Epoch 23/100
 - 0s - loss: 0.3806
Epoch 24/100
 - 0s - loss: 0.3736
Epoch 25/100
 - 0s - loss: 0.3604
Epoch 26/100
 - 0s - loss: 0.3510
Epoch 27/100
 - 0s - loss: 0.3391
Epoch 28/100
 - 0s - loss: 0.3339
Epoch 29/100
 - 0s - loss: 0.3217
Epoch 30/100
 - 0s - lo

In [5]:
# print number of species
print(species)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [9]:
pred = model.predict(x)
print(f"shape:{pred.shape}")
print(pred)

shape:(150, 3)
[[9.97036099e-01 2.96392501e-03 7.36863209e-11]
 [9.93743718e-01 6.25631027e-03 8.73866035e-10]
 [9.95243728e-01 4.75623598e-03 4.58504512e-10]
 [9.90622818e-01 9.37722530e-03 2.65831313e-09]
 [9.96928990e-01 3.07103246e-03 7.38520217e-11]
 [9.96127427e-01 3.87258315e-03 9.79121992e-11]
 [9.93796825e-01 6.20325282e-03 9.28645993e-10]
 [9.95584846e-01 4.41511581e-03 2.45260062e-10]
 [9.88709033e-01 1.12909907e-02 6.09625639e-09]
 [9.94096875e-01 5.90306008e-03 5.92375871e-10]
 [9.97789741e-01 2.21023778e-03 2.30286519e-11]
 [9.93130386e-01 6.86955592e-03 7.98793920e-10]
 [9.94289279e-01 5.71071915e-03 6.82795820e-10]
 [9.94327366e-01 5.67268347e-03 7.66404995e-10]
 [9.99080062e-01 9.19963408e-04 6.49316701e-13]
 [9.98688519e-01 1.31145597e-03 2.52806825e-12]
 [9.98081326e-01 1.91870774e-03 1.38985724e-11]
 [9.96568561e-01 3.43142310e-03 1.28269922e-10]
 [9.97260928e-01 2.73901201e-03 3.12202278e-11]
 [9.96864378e-01 3.13563459e-03 7.62056460e-11]
 [9.94779110e-01 5.220965

In [10]:
np.set_printoptions(suppress=True)

In [14]:
predict_classes = np.argmax(pred,axis=1)
expected_classes = np.argmax(y, axis=1)
print(species[predict_classes[1:10]])

['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa']


In [17]:
from sklearn.metrics import accuracy_score
correct = accuracy_score(expected_classes, predict_classes)
print(correct)

0.98


In [19]:
# save network to HDF5
import os
save_path = './Data/'
model.save(os.path.join(save_path,"network.h5"))

In [21]:
from keras.models import load_model
model2 = load_model(os.path.join(save_path,"network.h5"))
pred = model2.predict(x)